In [2]:
import matplotlib.pyplot as plt
import pymiediff as pmd
import torch
import numpy as np
import h5py
import torch.nn as nn
import torch.optim as optim


In [3]:
starting_wavelength = 380  # nm
ending_wavelength = 750  # nm

N_pt_test = 250

wl = torch.linspace(
    starting_wavelength,
    ending_wavelength,
    N_pt_test,
    dtype=torch.double,
    requires_grad=False,
)

k0 = 2 * torch.pi / wl

In [3]:
# define range of starting parameter combinations
r_c_min, r_c_max = 10.0, 20.0
r_s_min, r_s_max = 45.0, 55.0
n_c_min, n_c_max = 2.0 + 0.1j, 2.0 + 0.1j
n_s_min, n_s_max = 5.0 + 0.2j, 5.0 + 0.2j

# define number of starting parameter combinations
NumComb = 1000

r_c, r_s, n_c, n_s = pmd.seedComb(
    r_c_min,
    r_c_max,
    r_s_min,
    r_s_max,
    n_c_min,
    n_c_max,
    n_s_min,
    n_s_max,
    NumComb=NumComb,
)

In [4]:
# cross_section = np.zeros((NumComb, N_pt_test), dtype=np.float32)

# # Compute cross-section iteratively
# for i in range(NumComb):
#     cross_section[i] = pmd.farfield.cross_sections(
#         k0=k0,
#         r_c=r_c[i],
#         eps_c=n_c[i]**2,
#         r_s=r_s[i],
#         eps_s=n_s[i]**2,
#         eps_env=1,
#     )['q_sca']
#     if i % 50 == 0:
#         print(f"{i}/{NumComb}")

# # Save to HDF5 file
# h5_path = "dataset.h5"
# with h5py.File(h5_path, "w") as f:
#     f.create_dataset("r_c", data=r_c)
#     f.create_dataset("r_s", data=r_s)
#     f.create_dataset("n_c", data=n_c)
#     f.create_dataset("n_s", data=n_s)
#     f.create_dataset("cross_section", data=cross_section)

# print(f"Dataset saved to {h5_path}")

In [4]:
import h5py
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

class TorchStandardScaler:
    def __init__(self):
        self.mean = None
        self.std = None

    def fit(self, data):
        data = torch.tensor(data, dtype=torch.double)
        self.mean = data.mean(dim=0)
        self.std = data.std(dim=0)
        self.std[self.std == 0] = 1.0  # Avoid division by zero

    def transform(self, data):
        data = torch.tensor(data, dtype=torch.double)
        data = data.clone().detach().requires_grad_(True) #torch.tensor(data, dtype=torch.double) #
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        data = torch.tensor(data, dtype=torch.double)
        data = data.clone().detach().requires_grad_(True) #torch.tensor(data, dtype=torch.double)
        return data * self.std + self.mean

class CoreShellDataset(Dataset):
    def __init__(self, h5_file, fit_scalers=True, x_scaler=None, y_scaler=None):
        super().__init__()
        self.h5_file = h5_file

        # Open the file to get dataset sizes (but don't keep it open)
        with h5py.File(h5_file, "r") as f:
            self.length = len(f["r_c"])
            if fit_scalers:
                # Load all data to fit scalers
                x_data = np.stack([f["r_c"][:], f["r_s"][:], f["n_c"][:].real, f["n_c"][:].imag, f["n_s"][:].real, f["n_s"][:].imag], axis=1)
                y_data = f["cross_section"][:]

                self.x_scaler = TorchStandardScaler()
                self.y_scaler = TorchStandardScaler()

                self.x_scaler.fit(torch.tensor(x_data, dtype=torch.double))
                self.y_scaler.fit(torch.tensor(y_data, dtype=torch.double))
            else:
                self.x_scaler = x_scaler
                self.y_scaler = y_scaler

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        with h5py.File(self.h5_file, "r") as f:
            r_c = f["r_c"][idx]
            r_s = f["r_s"][idx]
            n_c_re = f["n_c"][idx].real
            n_c_im = f["n_c"][idx].imag
            n_s_re = f["n_s"][idx].real
            n_s_im = f["n_s"][idx].imag
            cross_section = f["cross_section"][idx]

        x = np.array([r_c, r_s, n_c_re, n_c_im, n_s_re, n_s_im]).reshape(1, -1)
        y = np.array(cross_section).reshape(1, -1)

        # Scale data using PyTorch scaler
        x = self.x_scaler.transform(x).flatten()
        y = self.y_scaler.transform(cross_section.reshape(1, -1)).flatten()

        return x, y

# Usage example
h5_path = "dataset.h5"
dataset = CoreShellDataset(h5_path)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

C:\Users\okcj1g19\AppData\Local\Temp\ipykernel_28992\570082199.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.double)


In [5]:
x_scaler = dataset.x_scaler
y_scaler = dataset.y_scaler

In [6]:
# Example of using the scalers on new data
new_x = np.array([[0.5, 0.7, 1.5, 2.0, 1.5, 2.0]])
scaled_x = x_scaler.transform(new_x)
original_x = x_scaler.inverse_transform(scaled_x)

new_y = np.random.rand(1, 250)  # Example y data with size 20
scaled_y = y_scaler.transform(new_y)
original_y = y_scaler.inverse_transform(scaled_y)

print("Scaled x:", scaled_x)
print("Original x:", original_x)
print("Scaled y:", scaled_y)
print("Original y:", original_y)


Scaled x: tensor([[ -4.9413, -17.4347,  -0.5000,   1.9000,  -3.5000,   1.8000]],
       dtype=torch.float64, grad_fn=<DivBackward0>)
Original x: tensor([[0.5000, 0.7000, 1.5000, 2.0000, 1.5000, 2.0000]], dtype=torch.float64,
       grad_fn=<AddBackward0>)
Scaled y: tensor([[-3.8395e+00, -3.7576e+00, -3.4057e+00, -3.9135e+00, -2.4388e+00,
         -2.4430e+00, -2.3811e+00, -2.3720e+00, -3.1418e+00, -2.1628e+00,
         -2.4879e+00, -2.9461e+00, -2.8374e+00, -3.2904e+00, -2.2517e+00,
         -3.2780e+00, -2.4634e+00, -2.3195e+00, -3.1900e+00, -3.6180e+00,
         -2.4164e+00, -2.0445e+00, -3.3728e+00, -2.2635e+00, -2.0780e+00,
         -2.7076e+00, -2.1123e+00, -4.2071e+00, -3.2919e+00, -4.1265e+00,
         -4.9286e+00, -3.6278e+00, -2.2657e+00, -3.2137e+00, -3.6881e+00,
         -5.0991e+00, -6.2507e+00, -5.8374e+00, -4.9642e+00, -7.1326e+00,
         -5.9458e+00, -7.9513e+00, -4.6823e+00, -6.9562e+00, -5.4900e+00,
         -3.1422e+00, -3.6788e+00, -7.1758e+00, -3.7533e+00, -6.2599

C:\Users\okcj1g19\AppData\Local\Temp\ipykernel_28992\570082199.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.double)


In [7]:
class MLP(nn.Module):
    def __init__(self, input_dim=250, output_dim=6, hidden_dim=128):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
test = torch.rand((32, 4))
test_shape = test.shape
print(test_shape)





torch.Size([32, 4])


In [8]:


def vector_batch(x_pred):


    batch_shape = x_pred.shape
    # [r_c, r_s, n_c_re, n_c_im, n_s_re, n_s_im]
    y_pred_temp = []

    for i in range(batch_shape[0]):
        x_batch = x_pred[i, :]
        r_c_i, r_s_i, n_c_re_i, n_c_im_i, n_s_re_i, n_s_im_i = x_batch[0], x_batch[1], x_batch[2], x_batch[3], x_batch[4], x_batch[5]

        #print("x", x_batch)

        y_batch = pmd.farfield.cross_sections(
            k0=k0,
            r_c=r_c_i,
            eps_c=(n_c_re_i+1j*n_c_im_i) ** 2,
            r_s=r_s_i,
            eps_s=(n_s_re_i+1j*n_s_im_i) ** 2,
            eps_env=1,
        )["q_sca"]



        # y_batch = y_batch.to(dtype=torch.double)  # Set dtype
        # y_batch[torch.isnan(y_batch)] = 0  # Replace NaNs with zero

        #print("y", y_batch[:10])
        y_pred_temp.append(y_batch)

    # print(y_pred_temp)

    y_pred = torch.stack(y_pred_temp)



    return y_pred


def train_model(model, dataloader, num_epochs=20, lr=1e-5, device="cuda" if torch.cuda.is_available() else "cpu"):
    model.to(device)
    model.double()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        total_loss = 0.0
        for x_batch, y_batch in dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            # Inverse model
            x_pred = model(y_batch)
            #print(x_pred.shape)
            # Forward model

            x_pred_scaled = x_scaler.inverse_transform(x_pred)

            y_pred = vector_batch(x_pred_scaled)#

            y_pred_scaled = y_scaler.transform(y_pred)
            # print("y test", y_batch[0, :10])
            # print("vVv Inverse network vVv")
            # print("x pred", x_pred_scaled[0, :])
            # print("vVv Forward Mie solver vVv")
            # print("y pred", y_pred[0, :10])

            loss = criterion(y_pred_scaled, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.6f}")

    print("Training complete!")

In [9]:
from torch.utils.data import DataLoader
h5_path = "dataset.h5"

train_dataset = CoreShellDataset(h5_path)
# scaler = train_dataset.scaler  # Save scaler for later use

# Create DataLoader for batch processing
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

C:\Users\okcj1g19\AppData\Local\Temp\ipykernel_28992\570082199.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.double)


Taking about 1min 30s on my laptop per epoc.

In [10]:
mlp_model = MLP()
train_model(mlp_model, dataloader, num_epochs=50, lr=1e-2)

C:\Users\okcj1g19\AppData\Local\Temp\ipykernel_28992\570082199.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.double)
C:\Users\okcj1g19\AppData\Local\Temp\ipykernel_28992\570082199.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.double)


KeyboardInterrupt: 

In [ ]:
# Get a sample from the dataset
x_sample, y_sample = train_dataset[0]  # Example input
x_sample = x_sample.unsqueeze(0)  # Add batch dimension

# Run inference
mlp_model.eval()
with torch.no_grad():
    y_pred = mlp_model(x_sample)

# Convert prediction back to original scale
y_pred_original = scaler.inverse_transform(y_pred.numpy())
